# BigGAN Hessian Computation
This repo tries to prove whether we could compute hessian of BigGAN (activation or image similarity) by forward finite difference method.

In [ ]:
from pytorch_pretrained_biggan import BigGAN, truncated_noise_sample
